# 필수 과제 (AI 챗봇 구현하기)

## 1.getpass를 이용하여 API키 불러오기

- getpass는 파이썬 내장모듈로 따로 설치할 필요없이 import 해서 바로사용 가능
- 패스워드를 입력할 때, 입력한 문자를 보여주지 않도록 해 준다.

In [2]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")


OpenAI API key 입력:  ········


## 2.모델 & 문서 로드하기

- 인공지능산업최신동향을 선택한 이유는 최신동향을 이해하는게 이 산업에 몸담기 위해 조금이라도 도움이 될까싶어서 이다.

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain.document_loaders import PyPDFLoader

# 모델 초기화
model = ChatOpenAI(model='gpt-4o-mini')

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("/Users/t2023-m0072/Desktop/task/인공지능산업최신동향_2024년11월호.pdf")

# 페이지 별 문서 로드
docs = loader.load()

## 3. 문서 청크로 나누기

In [ ]:
# 문서 청크로 나누기 방법 1

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits1 = text_splitter.split_documents(docs)

print(splits1[:9])

#### 청킹방식
- CharacterTextSplitter : 주어진 텍스트를 문자 단위로 분할하는 데 사용된다. 아래의 방식과 유사하지만 보다 구체적인 구분을 위해 사용자 정의 구분 기호를 정의할 수 있는 기능이 있다. 기본적으로 "\n\n", "\n", " ", ""와 같은 문자로 분할을 시도한다.  
#### 파라미터
- separator : 분할된 각 청크를 구분할 때 기준이 되는 문자열이다. 기본값은 "\n\n" 이다.
- chunk_size : 각 청크의 최대 길이이다. 여기서는 100으로 설정되어 있으므로, 최대 100자까지의 텍스트가 하나의 청크에 포함된다.
- chunk_overlap : 인접한 청크 사이에 중복으로 포함될 문자의 수이다. 여기서는 10으로 설정되어 있으므로, 각 청크들은 연결 부분에서 10자가 중복된다. 
- length_function : 청크의 길이를 계산하는 함수이다. 여기서는 len 함수가 사용되었으므로, 문자열의 길이를 기반으로 청크의 길이를 계산한다.
- is_separator_regex : 매개변수를 False로 설정하여 separator를 정규식이 아닌 일반 문자열로 처리합니다.
- text_splitter를 사용해 텍스트를 문서로 분활한다. 

In [ ]:
# 문서 청크로 나누기 방법 2

from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits2 = recursive_text_splitter.split_documents(docs)

print(splits2[:9])

#### 청킹방식
- 일반적인 텍스트에 권장되는 방식이다. 문자 목록을 매개변수로 받아 동작한다. 청크가 작아질 때까지 주어진 문자 목록의 순서대로 텍스트를 분할하려고 시도한다. 단락 -> 문자 -> 단어 순서로 재귀적으로 분할한다. 텍스트는 문자 목록 ["\n\n", "\n", " ", ""] 에 의해 분할 되고 청크 크기는 문자 수에 의해 측정된다. 
#### 파라미터는 위와 동일

## 4.벡터 임베딩 생성

In [13]:
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


## 5.FAISS 벡터 스토어 생성 & Retriever로 변환

In [36]:
import faiss
from langchain_community.vectorstores import FAISS


vectorstore = FAISS.from_documents(documents=splits1, embedding=embeddings)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

## 6.프롬프트 템플릿 정의

In [37]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an ai expert. Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

## 7.RAG 체인 구성

In [38]:
from langchain.chains import LLMChain

class SimplePassThrough:
    def invoke(self, inputs, **kwargs):
        return inputs

class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template
    
    def invoke(self, inputs):
        # 문서 내용을 텍스트로 변환
        if isinstance(inputs, list):
            context_text = "\n".join([doc.page_content for doc in inputs])
        else:
            context_text = inputs
        
        # 프롬프트 템플릿에 적용
        formatted_prompt = self.prompt_template.format_messages(
            context=context_text,
            question=inputs.get("question", "")
        )
        return formatted_prompt

# Retriever를 invoke() 메서드로 래핑하는 클래스 정의
class RetrieverWrapper:
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, inputs):
        if isinstance(inputs, dict):
            query = inputs.get("question", "")
        else:
            query = inputs
        # 검색 수행
        response_docs = self.retriever.get_relevant_documents(query)
        return response_docs

llm_chain = LLMChain(llm=model, prompt=contextual_prompt)

# RAG 체인 설정
rag_chain_debug = {
    "context": RetrieverWrapper(retriever),
    "prompt": ContextToPrompt(contextual_prompt),
    "llm": model
}

## 8.챗봇 구동

In [ ]:

# 챗봇 구동
while True:
    print("========================")
    query = input("질문을 입력하세요 : ")
    
    # 1. Retriever로 관련 문서 검색
    response_docs = rag_chain_debug["context"].invoke({"question": query})
    
    # 2. 문서를 프롬프트로 변환
    prompt_messages = rag_chain_debug["prompt"].invoke({
        "context": response_docs,
        "question": query
    })
    
    # 3. LLM으로 응답 생성
    response = rag_chain_debug["llm"].invoke(prompt_messages)
    
    print("\n답변:")
    print(response.content)